In [1]:
import numpy as np
import scipy as sp
import pandas as pd

In [2]:
from scipy.sparse import load_npz

import numpy as np
import csv
import os


def _load_csv(path):
    # A helper function to load the csv file.
    if not os.path.exists(path):
        raise Exception("The specified path {} does not exist.".format(path))
    # Initialize the data.
    data = {
        "user_id": [],
        "question_id": [],
        "is_correct": []
    }
    # Iterate over the row to fill in the data.
    with open(path, "r") as csv_file:
        reader = csv.reader(csv_file)
        for row in reader:
            try:
                data["question_id"].append(int(row[0]))
                data["user_id"].append(int(row[1]))
                data["is_correct"].append(int(row[2]))
            except ValueError:
                # Pass first row.
                pass
            except IndexError:
                # is_correct might not be available.
                pass
    return data


def load_train_sparse(root_dir="/data"):
    """ Load the training data as a spare matrix representation.

    :param root_dir: str
    :return: 2D sparse matrix
    """
    path = os.path.join(root_dir, "train_sparse.npz")
    if not os.path.exists(path):
        raise Exception("The specified path {} "
                        "does not exist.".format(os.path.abspath(path)))
    matrix = load_npz(path)
    return matrix


def load_train_csv(root_dir="/data"):
    """ Load the training data as a dictionary.

    :param root_dir: str
    :return: A dictionary {user_id: list, question_id: list, is_correct: list}
        WHERE
        user_id: a list of user id.
        question_id: a list of question id.
        is_correct: a list of binary value indicating the correctness of
        (user_id, question_id) pair.
    """
    path = os.path.join(root_dir, "train_data.csv")
    return _load_csv(path)


def load_valid_csv(root_dir="/data"):
    """ Load the validation data as a dictionary.

    :param root_dir: str
    :return: A dictionary {user_id: list, question_id: list, is_correct: list}
        WHERE
        user_id: a list of user id.
        question_id: a list of question id.
        is_correct: a list of binary value indicating the correctness of
        (user_id, question_id) pair.
    """
    path = os.path.join(root_dir, "valid_data.csv")
    return _load_csv(path)


def load_public_test_csv(root_dir="/data"):
    """ Load the test data as a dictionary.

    :param root_dir: str
    :return: A dictionary {user_id: list, question_id: list, is_correct: list}
        WHERE
        user_id: a list of user id.
        question_id: a list of question id.
        is_correct: a list of binary value indicating the correctness of
        (user_id, question_id) pair.
    """
    path = os.path.join(root_dir, "test_data.csv")
    return _load_csv(path)


def load_private_test_csv(root_dir="/data"):
    """ Load the private test data as a dictionary.

    :param root_dir: str
    :return: A dictionary {user_id: list, question_id: list, is_correct: list}
        WHERE
        user_id: a list of user id.
        question_id: a list of question id.
        is_correct: an empty list.
    """
    path = os.path.join(root_dir, "private_test_data.csv")
    return _load_csv(path)


def save_private_test_csv(data, file_name="private_test_result.csv"):
    """ Save the private test data as a csv file.

    This should be your submission file to Kaggle.
    :param data: A dictionary {user_id: list, question_id: list, is_correct: list}
        WHERE
        user_id: a list of user id.
        question_id: a list of question id.
        is_correct: a list of binary value indicating the correctness of
        (user_id, question_id) pair.
    :param file_name: str
    :return: None
    """
    if not isinstance(data, dict):
        raise Exception("Data must be a dictionary.")
    cur_id = 1
    valid_id = ["0", "1"]
    with open(file_name, "w") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["id", "is_correct"])
        for i in range(len(data["user_id"])):
            if str(int(data["is_correct"][i])) not in valid_id:
                raise Exception("Your data['is_correct'] is not in a valid format.")
            writer.writerow([str(cur_id), str(int(data["is_correct"][i]))])
            cur_id += 1
    return


def evaluate(data, predictions, threshold=0.5):
    """ Return the accuracy of the predictions given the data.

    :param data: A dictionary {user_id: list, question_id: list, is_correct: list}
    :param predictions: list
    :param threshold: float
    :return: float
    """
    if len(data["is_correct"]) != len(predictions):
        raise Exception("Mismatch of dimensions between data and prediction.")
    if isinstance(predictions, list):
        predictions = np.array(predictions).astype(np.float64)
    return (np.sum((predictions >= threshold) == data["is_correct"])
            / float(len(data["is_correct"])))


def sparse_matrix_evaluate(data, matrix, threshold=0.5):
    """ Given the sparse matrix represent, return the accuracy of the prediction on data.

    :param data: A dictionary {user_id: list, question_id: list, is_correct: list}
    :param matrix: 2D matrix
    :param threshold: float
    :return: float
    """
    total_prediction = 0
    total_accurate = 0
    for i in range(len(data["is_correct"])):
        cur_user_id = data["user_id"][i]
        cur_question_id = data["question_id"][i]
        if matrix[cur_user_id, cur_question_id] >= threshold and data["is_correct"][i]:
            total_accurate += 1
        if matrix[cur_user_id, cur_question_id] < threshold and not data["is_correct"][i]:
            total_accurate += 1
        total_prediction += 1
    return total_accurate / float(total_prediction)


def sparse_matrix_predictions(data, matrix, threshold=0.5):
    """ Given the sparse matrix represent, return the predictions.

    This function can be used for submitting Kaggle competition.

    :param data: A dictionary {user_id: list, question_id: list, is_correct: list}
    :param matrix: 2D matrix
    :param threshold: float
    :return: list
    """
    predictions = []
    for i in range(len(data["user_id"])):
        cur_user_id = data["user_id"][i]
        cur_question_id = data["question_id"][i]
        if matrix[cur_user_id, cur_question_id] >= threshold:
            predictions.append(1.)
        else:
            predictions.append(0.)
    return predictions


### KNN

In [13]:
train_data = load_train_csv("../data")
val_data = load_valid_csv("../data")
test_data = load_public_test_csv("../data")

In [17]:
np.array(train_data['user_id'])

array([488, 355, 123, ...,  49, 308, 514])

In [37]:
def bootstrap_idx(data, matrix, num_resamples):
    resampled_data = []
    resampled_matrix = []
    
    for idx in range(num_resamples):
        random_idx = np.random.randint(0, len(matrix), len(matrix))
        random_idx = np.arange(0, len(matrix))
        sampled_dict = {}
        sampled_dict['user_id'] = list(np.array(data['user_id'])[random_idx])
        sampled_dict['question_id'] = list(np.array(data['question_id'])[random_idx])
        sampled_dict['is_correct'] = list(np.array(data['is_correct'])[random_idx])
        resampled_data.append(sampled_dict)
        resampled_matrix.append(matrix[random_idx])
    return resampled_data, resampled_matrix
        
    

In [35]:
a, b = bootstrap_idx(train_data, sparse_matrix, 3)
b[1]

[510  53 326 294 214 130  31  43 273 476 309 251  92  96  51 371 305 453
 172  21 502  91 343  77  61 108 167 191 382 403 335 142 201 502 157 431
 273 377 244 540 231 424 402   3 155  95 401 425 333  78  49 540 183 314
 360  56 504 473 189 370 229 196 466 143 353 528   1  97 141 183 192 372
 532 424 417 106 220 117 151 331 194  57 302 172 295 131 155 342 269  89
 432 224 359   6 124  26 280 243 379 256 374 361 198 253  25  88 454 287
 312 521 105 427 109 524 138 134 528 158 455 269 161 186 169 318 153 491
 405 413 283 239  82  14   5 111 456 418 311 394 161 264 101 323  72 135
  62 152 534 396 499 318 138 187 112 473 290   6 466 284 276   2 507 206
 106 453  51  93 321 538 339 258 337 129  93 450 507 215 239   8 149 333
 509 243 441  38 282  43 424 220 125 475 234  64 320 311 151 197 540 222
 240 225 160  87 191 119 279 509 152 501 231 120  20 428  29 191 318 196
 152 502  10 102 482 304 444   9  25 356  48  15 347  54 122 305 224  29
 483  29 155 100 270 301 298 536 503 334 297 204  9

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [14]:
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
# from utils import *


def knn_impute_by_user(matrix, valid_data, k):
    """ Fill in the missing values using k-Nearest Neighbors based on
    student similarity. Return the accuracy on valid_data.

    See https://scikit-learn.org/stable/modules/generated/sklearn.
    impute.KNNImputer.html for details.

    :param matrix: 2D sparse matrix
    :param valid_data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param k: int
    :return: float
    """
    nbrs = KNNImputer(n_neighbors=k)
    # We use NaN-Euclidean distance measure.
    mat = nbrs.fit_transform(matrix)
    acc = sparse_matrix_evaluate(valid_data, mat)
    # print("Validation Accuracy: {}".format(acc))
    return acc


def knn_impute_by_item(matrix, valid_data, k):
    """ Fill in the missing values using k-Nearest Neighbors based on
    question similarity. Return the accuracy on valid_data.

    :param matrix: 2D sparse matrix
    :param valid_data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param k: int
    :return: float
    """
    #####################################################################
    # TODO:                                                             #
    # Implement the function as described in the docstring.             #
    #####################################################################
    nbrs = KNNImputer(n_neighbors=k)
    # We use NaN-Euclidean distance measure.
    mat = nbrs.fit_transform(matrix.T).T
    acc = sparse_matrix_evaluate(valid_data, mat)
    # print("Validation Accuracy: {}".format(acc))
    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################
    return acc


def plot_accuracy(k_list, acc, filename, title):
    plt.plot(k_list, acc)
    plt.xlabel("k")
    plt.xticks(k_list)
    plt.grid(axis='x', color='0.95')
    plt.title(title)
    plt.savefig("../figs/" + filename + ".png")
    plt.show()


def main():
    sparse_matrix = load_train_sparse("../data").toarray()
    val_data = load_valid_csv("../data")
    test_data = load_public_test_csv("../data")

    print("Sparse matrix:")
    print(sparse_matrix)
    print("Shape of sparse matrix:")
    print(sparse_matrix.shape)

    #####################################################################
    # TODO:                                                             #
    # Compute the validation accuracy for each k. Then pick k* with     #
    # the best performance and report the test accuracy with the        #
    # chosen k*.                                                        #
    #####################################################################

    k_list = [1, 6, 11, 16, 21, 26]
    train_acc_by_user, val_acc_by_item = [], []

    for k in k_list:
        print("K: " + str(k))

        res = knn_impute_by_user(sparse_matrix, val_data, k)
        print("User based collaborative filtering: " + str(res))
        train_acc_by_user.append(res)

        res = knn_impute_by_item(sparse_matrix, val_data, k)
        print("Item based collaborative filtering: " + str(res))
        val_acc_by_item.append(res)

    plot_accuracy(k_list, train_acc_by_user, "Q1a", "User based validation accuracy")
    plot_accuracy(k_list, val_acc_by_item, "Q1c", "Item based validation accuracy")

    # Final test accuracy
    best_k = 11
    print(knn_impute_by_user(sparse_matrix, test_data, best_k))

    best_k = 21
    print(knn_impute_by_item(sparse_matrix, test_data, best_k))

    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################


In [15]:
k = 5
nbrs = KNNImputer(n_neighbors=k)


In [18]:
a = nbrs.fit_transform(sparse_matrix)

In [37]:
i = 0
non_null_idx = np.argwhere(np.isnan(sparse_matrix[i, :]))


array([  1000.        ,   1274.2749857 ,   1623.77673919,   2069.13808111,
         2636.65089873,   3359.81828628,   4281.33239872,   5455.59478117,
         6951.92796178,   8858.6679041 ,  11288.37891685,  14384.49888288,
        18329.80710832,  23357.2146909 ,  29763.51441631,  37926.90190732,
        48329.30238572,  61584.8211066 ,  78475.99703515, 100000.        ])

In [31]:
a

array([[0.2       , 0.8       , 0.6       , ..., 0.4       , 0.6       ,
        0.4       ],
       [0.4       , 0.        , 0.4       , ..., 0.4       , 0.6       ,
        0.6       ],
       [0.4       , 0.8       , 1.        , ..., 0.4       , 0.6       ,
        0.8       ],
       ...,
       [0.8       , 0.53846154, 0.4       , ..., 0.2       , 0.8       ,
        0.5       ],
       [0.4       , 0.6       , 0.4       , ..., 0.2       , 0.6       ,
        0.8       ],
       [0.2       , 0.4       , 0.4       , ..., 0.45454545, 0.6       ,
        0.2       ]])

In [80]:
np.unique(np.random.randint(1, 100, 100), return_counts=True)

(array([ 1,  3,  4,  6,  8, 13, 14, 15, 16, 18, 21, 24, 25, 26, 29, 30, 33,
        36, 38, 40, 41, 42, 45, 47, 48, 49, 51, 52, 53, 55, 56, 57, 58, 61,
        62, 64, 65, 66, 67, 69, 70, 73, 76, 77, 79, 80, 82, 83, 84, 85, 86,
        88, 90, 91, 92, 93, 94, 95, 97, 99]),
 array([2, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4, 4, 1, 1, 1, 3, 1, 2, 1,
        1, 3, 1, 2, 1, 1, 3, 2, 1, 1, 2, 3, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 1, 1, 3, 1, 1, 1, 2, 2, 3, 1, 2, 1, 1], dtype=int64))

In [96]:
np.repeat(sparse_matrix, 3, axis=0).reshape(3, sparse_matrix.shape[0], sparse_matrix.shape[1])

(3, 542, 1774)

In [128]:
def bootstrap_idx(data, num_resamples):
    idx = np.random.randint(0, len(data), (num_resamples, len(data)))
    return idx, data[idx]

In [129]:
i, d = bootstrap_idx(sparse_matrix, 3)

In [141]:
np.allclose(sparse_matrix[395], d[0][0])

False

In [142]:
sparse_matrix

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan,  0., nan, ..., nan, nan, nan],
       [nan, nan,  1., ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [143]:
# np.concatenate([train_matrix, train_matrix, train_matrix]).reshape(3, len(train_matrix), -1)

In [38]:
[2, 3, 4] == np.array([2,3,4])

array([ True,  True,  True])

In [147]:
d[0][4]

array([ 0., nan, nan, ..., nan, nan, nan])

In [151]:
sparse_matrix[373]

array([ 0., nan, nan, ..., nan, nan, nan])

In [39]:

def bootstrap_idx(data, matrix, num_resamples):
    resampled_data = []
    resampled_matrix = []

    for idx in range(num_resamples):
        random_idx = np.random.randint(0, len(matrix), len(matrix))
        random_idx = np.arange(0, len(matrix))
        # if idx == 0:
        #     random_idx = np.repeat(np.arange(0, len(matrix)//2), 2)
        # elif idx == 1:
        #     random_idx = np.repeat(np.arange(len(matrix//2), len(matrix)), 2)
        sampled_dict = {}
        sampled_dict['user_id'] = list(np.array(data['user_id'])[random_idx])
        sampled_dict['question_id'] = list(np.array(data['question_id'])[random_idx])
        sampled_dict['is_correct'] = list(np.array(data['is_correct'])[random_idx])
        resampled_data.append(sampled_dict)
        resampled_matrix.append(matrix[random_idx])
    return resampled_data, resampled_matrix

In [41]:
x, y = bootstrap_idx(train_data, sparse_matrix, 3)

In [53]:
for j in range(3):
    for i in range(len(x[j]["user_id"])):
        cur_user_id = x[j]["user_id"][i]
        cur_question_id = x[j]["question_id"][i]
        if x[j]["is_correct"][i] != y[j][cur_user_id, cur_question_id]:
            print(x[j]["is_correct"])

In [55]:
print(len(train_data["user_id"]))

56688


In [59]:
np.unique(np.array(train_data["user_id"]), return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [71]:
np.random.seed(1)
theta = np.random.rand(sparse_matrix.shape[0], 1)
beta = np.random.rand(sparse_matrix.shape[1], 1)

In [73]:
def update_theta_beta_2(data, lr, theta, beta):
    """ Update theta and beta using gradient descent.

    You are using alternating gradient descent. Your update should look:
    for i in iterations ...
        theta <- new_theta
        beta <- new_beta

    You may optionally replace the function arguments to receive a matrix.

    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param lr: float
    :param theta: Vector
    :param beta: Vector
    :return: tuple of vectors
    """
    #####################################################################
    # TODO:                                                             #
    # Implement the function as described in the docstring.             #
    #####################################################################
    # temp = (data == 1).astype(int)
    for i in range(len(theta)):
        non_null_idx = np.where(~np.isnan(data[i, :]))[0]
        temp = sigmoid(theta[i] - beta)

        # print(data[i, :].shape)
        # print(temp.shape)
        # print(temp[non_null_idx].shape)
        # print(non_null_idx.shape)
        theta[i] += lr * (np.sum(data[i, :][non_null_idx]) - np.sum(temp[non_null_idx]))

    for j in range(len(beta)):
        non_null_idx = np.where(~np.isnan(data[:, j]))[0]
        temp = sigmoid(theta - beta[j])
        beta[j] += lr * (-np.sum(data[:, j][non_null_idx]) + np.sum(temp[non_null_idx]))

    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################
    return theta, beta

In [87]:

def update_theta_beta(data, lr, theta, beta):
    """ Update theta and beta using gradient descent.

    You are using alternating gradient descent. Your update should look:
    for i in iterations ...
        theta <- new_theta
        beta <- new_beta

    You may optionally replace the function arguments to receive a matrix.

    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param lr: float
    :param theta: Vector
    :param beta: Vector
    :return: tuple of vectors
    """
    #####################################################################
    # TODO:                                                             #
    # Implement the function as described in the docstring.             #
    #####################################################################
    
    u_id_arr = np.array(data["user_id"])
    q_id_arr = np.array(data["question_id"])
    c_id_arr = np.array(data["is_correct"])

    for i in range(len(theta)):
        theta[i] -= lr * (np.sum(sigmoid(theta[i] - beta)[q_id_arr[u_id_arr == i]]) - np.sum(c_id_arr[u_id_arr == i]))

    for j in range(len(beta)):
        beta[j] -= lr * (np.sum(c_id_arr[q_id_arr == j]) - np.sum(sigmoid(theta - beta[j])[u_id_arr[q_id_arr == j]]))

    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################
    return theta, beta

In [77]:
data = train_data
u_id_arr = np.array(data["user_id"])
q_id_arr = np.array(data["question_id"])
c_id_arr = np.array(data["is_correct"])


In [85]:
sigmoid(theta[0] - beta)[q_id_arr[u_id_arr == 0]]

(305, 1)

In [80]:

def sigmoid(x):
    """ Apply sigmoid function.
    """
    return np.exp(x) / (1 + np.exp(x))


In [99]:
theta = np.zeros(shape=(542))
beta = np.zeros(shape=(1774))

x, y = update_theta_beta(train_data, 0.01, theta, beta)

In [98]:
theta = np.zeros(shape=(542))
beta = np.zeros(shape=(1774))

z, w = update_theta_beta_2(sparse_matrix, 0.01, theta, beta)

In [103]:
np.allclose(w, y)

True

In [104]:
from sklearn.impute import KNNImputer

In [112]:
new_imputer = KNNImputer(n_neighbors=5, weights="distance")

In [114]:
new_imputer.fit_transform(sparse_matrix)

array([[0.19292106, 0.77681885, 0.5       , ..., 0.34613912, 1.        ,
        0.42673644],
       [0.44157109, 0.        , 0.66666667, ..., 0.        , 0.66666667,
        0.        ],
       [0.        , 0.5       , 1.        , ..., 0.        , 1.        ,
        0.8025339 ],
       ...,
       [1.        , 0.53846154, 1.        , ..., 0.        , 0.80667439,
        0.5       ],
       [0.25      , 0.6       , 0.5       , ..., 0.        , 0.        ,
        0.8       ],
       [0.2       , 0.4       , 0.4       , ..., 0.45454545, 0.        ,
        0.        ]])

In [116]:
len(data['user_id'])

56688